# Realized Variance (RV) Prediction - Models

**Objective:** Predict next-day Realized Variance (RV) using engineered features from high-frequency volatility data

**Dataset:**
- 30 Dow Jones stocks (2003-2024)
- 74 engineered features across 8 categories
- Train: 2003-2018 | Validation: 2019-2021 | Test: 2022-2024

**Approach:** Regression models (XGBoost, LightGBM, Random Forest)

In [20]:
# Import Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)


In [21]:
# Load data from Feature Engineering output
data_path = '../Feature Engineering Group 6/data/engineered/'

train_df = pd.read_parquet(data_path + 'rv_features_train.parquet')
val_df = pd.read_parquet(data_path + 'rv_features_val.parquet')
test_df = pd.read_parquet(data_path + 'rv_features_test.parquet')

print(f"\nTotal Samples: {len(train_df) + len(val_df) + len(test_df):,}")
print(f"Total Features: {train_df.shape[1]}")

# Display sample
print("\nSample of training data:")
print(train_df.head())



Total Samples: 153,575
Total Features: 74

Sample of training data:
        Date Ticker       RV     BPV    Good     Bad        RQ    RV_5  \
0 2003-01-02   AAPL   8.3082  6.2018  5.4820  2.8262  263.0252  6.4939   
1 2003-01-03   AAPL   6.5682  5.3314  3.3633  3.2048   98.4764  6.5745   
2 2003-01-06   AAPL   7.3444  6.1792  3.8547  3.4897  301.8829  5.9923   
3 2003-01-07   AAPL  10.0133  9.1303  5.5418  4.4715  215.2682  9.5007   
4 2003-01-08   AAPL   6.0982  4.9211  3.2482  2.8500   97.9670  4.9405   

    BPV_5  Good_5   Bad_5      RQ_5  RV_lag1  RV_lag5  RV_lag10  RV_lag20  \
0  3.7720  5.1023  1.3916  152.7294      NaN      NaN       NaN       NaN   
1  5.8933  3.6380  2.9365   62.0543   8.3082      NaN       NaN       NaN   
2  4.4432  3.8791  2.1132  180.2133   6.5682      NaN       NaN       NaN   
3  7.1200  5.7827  3.7179  197.3819   7.3444      NaN       NaN       NaN   
4  5.4333  2.4732  2.4674   35.7264  10.0133      NaN       NaN       NaN   

   RV_roll_mean_5  RV_r

In [22]:
# Define feature groups
identifier_cols = ['Date', 'Ticker']
target_col = 'RV'

# Get all feature columns (exclude identifiers and target)
feature_cols = [c for c in train_df.columns if c not in identifier_cols + [target_col]]

print(f"Total features for modeling: {len(feature_cols)}")

# Categorize features for analysis
temporal_features = [c for c in feature_cols if 'lag' in c or 'roll' in c or 'momentum' in c or 'volatility' in c]
decomp_features = [c for c in feature_cols if 'Good' in c or 'Bad' in c or 'jump' in c or 'decomp' in c]
market_features = [c for c in feature_cols if 'market' in c or 'rank' in c or 'vs_market' in c]
freq_features = [c for c in feature_cols if 'freq' in c or 'microstructure' in c]
calendar_features = ['year', 'month', 'quarter', 'day_of_week', 'day_of_month', 'week_of_year', 
                     'is_monday', 'is_friday', 'is_month_end']
original_measures = ['BPV', 'Good', 'Bad', 'RQ', 'RV_5', 'BPV_5', 'Good_5', 'Bad_5', 'RQ_5']


Total features for modeling: 71


## 3. Feature Scaling & Transformation

**Rationale:**
- Volatility measures are heavily right-skewed and dominated by extreme events (COVID crash, rate hikes)

**Approach:**
1. **Log transformation** for right-skewed volatility measures (RV, BPV, RQ, Good, Bad, etc.)
2. **StandardScaler** for all continuous features to normalize scale
3. **No scaling** for binary/categorical features


In [23]:
# Import scaling libraries
from sklearn.preprocessing import StandardScaler

# Prepare X and y for each dataset (before scaling)
X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()

X_val = val_df[feature_cols].copy()
y_val = val_df[target_col].copy()

X_test = test_df[feature_cols].copy()
y_test = test_df[target_col].copy()

# Handle missing values first (lag features at start of time series)
missing_counts = X_train.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)

if len(missing_counts) > 0:
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)
    X_test = X_test.fillna(0)

log_transform_features = [
    # Original volatility measures (1-min) - always positive
    'BPV', 'Good', 'Bad', 'RQ',
    # 5-min measures - always positive
    'RV_5', 'BPV_5', 'Good_5', 'Bad_5', 'RQ_5',
    # Lagged features - always positive (RV is always positive)
    'RV_lag1', 'RV_lag5', 'RV_lag10', 'RV_lag20',
    # Rolling statistics - always positive
    'RV_roll_mean_5', 'RV_roll_std_5', 'RV_roll_min_5', 'RV_roll_max_5',
    'RV_roll_mean_20', 'RV_roll_std_20', 'RV_roll_min_20', 'RV_roll_max_20',
    'RV_roll_mean_60', 'RV_roll_std_60', 'RV_roll_min_60', 'RV_roll_max_60',
    # Market-wide measures - always positive
    'market_RV_mean', 'market_RV_median', 'market_RV_std', 'market_RV_min', 'market_RV_max',
    # Other positive volatility features
    'RV_volatility_20', 'jump_intensity', 'market_dispersion'
    # EXCLUDED: 'RV_momentum_5', 'RV_momentum_20' (can be negative)
    # EXCLUDED: 'RV_zscore' (can be negative)
    # EXCLUDED: 'microstructure_noise' (can be negative)
    # EXCLUDED: 'jump_diff_freq' (can be negative)
]

# Filter to only include features that exist in the dataset
log_transform_features = [f for f in log_transform_features if f in feature_cols]

# Apply log1p transformation
for col in log_transform_features:
    X_train[col] = np.log1p(X_train[col])
    X_val[col] = np.log1p(X_val[col])
    X_test[col] = np.log1p(X_test[col])

# Verify no NaN values
nan_count = X_train.isnull().sum().sum()
if nan_count > 0:
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)
    X_test = X_test.fillna(0)

no_scale_features = [
    'year', 'month', 'quarter', 'day_of_week', 'day_of_month', 'week_of_year',
    'is_monday', 'is_friday', 'is_month_end',
    'jump_indicator',  # Binary
    'RV_is_missing', 'BPV_is_missing', 'Good_is_missing', 'Bad_is_missing',  # Binary
]

# Features to scale
scale_features = [f for f in feature_cols if f not in no_scale_features]

scaler = StandardScaler()
scaler.fit(X_train[scale_features])

# Transform all datasets
X_train[scale_features] = scaler.transform(X_train[scale_features])
X_val[scale_features] = scaler.transform(X_val[scale_features])
X_test[scale_features] = scaler.transform(X_test[scale_features])

y_train_original = y_train.copy()
y_val_original = y_val.copy()
y_test_original = y_test.copy()

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)

print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nLog-transformed features: {len(log_transform_features)}")
print(f"StandardScaled features: {len(scale_features)}")
print(f"Unscaled features: {len(no_scale_features)}")
print(f"Target variable (RV): log1p transformed")


X_train: (114058, 71)
X_val: (22657, 71)
X_test: (16860, 71)

Log-transformed features: 33
StandardScaled features: 57
Unscaled features: 14
Target variable (RV): log1p transformed


In [24]:
# Initialize Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# Train model
rf_model.fit(X_train, y_train)

# Make predictions (in log-space)
y_train_pred_rf_log = rf_model.predict(X_train)
y_val_pred_rf_log = rf_model.predict(X_val)

# Inverse transform predictions back to original scale
y_train_pred_rf = np.expm1(y_train_pred_rf_log)
y_val_pred_rf = np.expm1(y_val_pred_rf_log)

# Evaluate on all sets (using original scale)
def evaluate_model(y_true_original, y_pred_original):
    """Evaluate model performance on original (non-transformed) scale"""
    rmse = np.sqrt(mean_squared_error(y_true_original, y_pred_original))
    mae = mean_absolute_error(y_true_original, y_pred_original)
    r2 = r2_score(y_true_original, y_pred_original)
    mape = mean_absolute_percentage_error(y_true_original, y_pred_original) * 100
    
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

rf_train_metrics = evaluate_model(y_train_original, y_train_pred_rf)
rf_val_metrics = evaluate_model(y_val_original, y_val_pred_rf)


In [25]:
# Initialize XGBoost
xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.1,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# Make predictions (in log-space)
y_train_pred_xgb_log = xgb_model.predict(X_train)
y_val_pred_xgb_log = xgb_model.predict(X_val)

# Inverse transform predictions back to original scale
y_train_pred_xgb = np.expm1(y_train_pred_xgb_log)
y_val_pred_xgb = np.expm1(y_val_pred_xgb_log)

# Evaluate
xgb_train_metrics = evaluate_model(y_train_original, y_train_pred_xgb)
xgb_val_metrics = evaluate_model(y_val_original, y_val_pred_xgb)


In [26]:
# Initialize LightGBM
lgbm_model = LGBMRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgbm_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[])

# Make predictions (in log-space)
y_train_pred_lgbm_log = lgbm_model.predict(X_train)
y_val_pred_lgbm_log = lgbm_model.predict(X_val)

# Inverse transform predictions back to original scale
y_train_pred_lgbm = np.expm1(y_train_pred_lgbm_log)
y_val_pred_lgbm = np.expm1(y_val_pred_lgbm_log)

# Evaluate
lgbm_train_metrics = evaluate_model(y_train_original, y_train_pred_lgbm)
lgbm_val_metrics = evaluate_model(y_val_original, y_val_pred_lgbm)


In [27]:
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'LightGBM'],
    'Train_RMSE': [rf_train_metrics['RMSE'], xgb_train_metrics['RMSE'], lgbm_train_metrics['RMSE']],
    'Train_R2': [rf_train_metrics['R2'], xgb_train_metrics['R2'], lgbm_train_metrics['R2']],
    'Train_MAE': [rf_train_metrics['MAE'], xgb_train_metrics['MAE'], lgbm_train_metrics['MAE']],
    'Val_RMSE': [rf_val_metrics['RMSE'], xgb_val_metrics['RMSE'], lgbm_val_metrics['RMSE']],
    'Val_R2': [rf_val_metrics['R2'], xgb_val_metrics['R2'], lgbm_val_metrics['R2']],
    'Val_MAE': [rf_val_metrics['MAE'], xgb_val_metrics['MAE'], lgbm_val_metrics['MAE']]
})

print("\nPerformance Comparison (Training & Validation):")
print(comparison_df.to_string(index=False))

# Find best model based on validation R2
best_model_idx = comparison_df['Val_R2'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
print(f"\nBest Model (by Validation R²): {best_model_name}")
print(f"  Training R²: {comparison_df.loc[best_model_idx, 'Train_R2']:.4f}")
print(f"  Validation R²: {comparison_df.loc[best_model_idx, 'Val_R2']:.4f}")
print(f"  Validation RMSE: {comparison_df.loc[best_model_idx, 'Val_RMSE']:.4f}")



Performance Comparison (Training & Validation):
        Model  Train_RMSE  Train_R2  Train_MAE  Val_RMSE  Val_R2  Val_MAE
Random Forest      1.1079    0.9706     0.0285    0.3998  0.9967   0.0379
      XGBoost      1.1414    0.9688     0.0700    1.1286  0.9734   0.1086
     LightGBM      1.2583    0.9621     0.0644    1.1307  0.9733   0.1064

Best Model (by Validation R²): Random Forest
  Training R²: 0.9706
  Validation R²: 0.9967
  Validation RMSE: 0.3998


## Hyperparameter Tuning - Random Forest


In [28]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [50, 100],
    'max_depth': [10, 15],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5],
    'max_features': ['sqrt', 'log2']
}

# Initialize base model
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=0)

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=10,
    cv=2,
    scoring='r2',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X_train, y_train)

print("\nBest parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nBest CV R² score: {random_search.best_score_:.4f}")


Fitting 2 folds for each of 10 candidates, totalling 20 fits

Best parameters found:
  n_estimators: 100
  min_samples_split: 5
  min_samples_leaf: 2
  max_features: sqrt
  max_depth: 15

Best CV R² score: 0.9989


## Final Model Evaluation


In [29]:
best_rf_model = random_search.best_estimator_

y_train_pred_tuned_log = best_rf_model.predict(X_train)
y_val_pred_tuned_log = best_rf_model.predict(X_val)
y_test_pred_tuned_log = best_rf_model.predict(X_test)

y_train_pred_tuned = np.expm1(y_train_pred_tuned_log)
y_val_pred_tuned = np.expm1(y_val_pred_tuned_log)
y_test_pred_tuned = np.expm1(y_test_pred_tuned_log)

# Evaluate on all sets
train_metrics_tuned = evaluate_model(y_train_original, y_train_pred_tuned)
val_metrics_tuned = evaluate_model(y_val_original, y_val_pred_tuned)
test_metrics_tuned = evaluate_model(y_test_original, y_test_pred_tuned)

print("\nTraining Set Performance:")
print(f"  RMSE: {train_metrics_tuned['RMSE']:.4f}")
print(f"  MAE:  {train_metrics_tuned['MAE']:.4f}")
print(f"  R²:   {train_metrics_tuned['R2']:.4f}")
print(f"  MAPE: {train_metrics_tuned['MAPE']:.2f}%")

print("\nValidation Set Performance:")
print(f"  RMSE: {val_metrics_tuned['RMSE']:.4f}")
print(f"  MAE:  {val_metrics_tuned['MAE']:.4f}")
print(f"  R²:   {val_metrics_tuned['R2']:.4f}")
print(f"  MAPE: {val_metrics_tuned['MAPE']:.2f}%")

print("\nTest Set Performance:")
print(f"  RMSE: {test_metrics_tuned['RMSE']:.4f}")
print(f"  MAE:  {test_metrics_tuned['MAE']:.4f}")
print(f"  R²:   {test_metrics_tuned['R2']:.4f}")
print(f"  MAPE: {test_metrics_tuned['MAPE']:.2f}%")

print("\n" + "="*70)
print("COMPARISON: BASELINE vs TUNED")
print("="*70)

comparison_final = pd.DataFrame({
    'Model': ['Baseline RF', 'Tuned RF'],
    'Train_R2': [rf_train_metrics['R2'], train_metrics_tuned['R2']],
    'Train_RMSE': [rf_train_metrics['RMSE'], train_metrics_tuned['RMSE']],
    'Val_R2': [rf_val_metrics['R2'], val_metrics_tuned['R2']],
    'Val_RMSE': [rf_val_metrics['RMSE'], val_metrics_tuned['RMSE']],
    'Test_R2': [None, test_metrics_tuned['R2']],
    'Test_RMSE': [None, test_metrics_tuned['RMSE']]
})

print("\nPerformance Comparison:")
print(comparison_final.to_string(index=False))
print("="*70)



Training Set Performance:
  RMSE: 1.2510
  MAE:  0.0361
  R²:   0.9625
  MAPE: 1.00%

Validation Set Performance:
  RMSE: 0.4959
  MAE:  0.0561
  R²:   0.9949
  MAPE: 1.57%

Test Set Performance:
  RMSE: 1.0132
  MAE:  0.0468
  R²:   0.8632
  MAPE: 1.42%

COMPARISON: BASELINE vs TUNED

Performance Comparison:
      Model  Train_R2  Train_RMSE  Val_R2  Val_RMSE  Test_R2  Test_RMSE
Baseline RF    0.9706      1.1079  0.9967    0.3998      NaN        NaN
   Tuned RF    0.9625      1.2510  0.9949    0.4959   0.8632     1.0132
